In [1]:
import numpy as np
import pandas as pd
from random import shuffle

## Helper Functions

In [2]:
def read_bank_note(test=False, data_frame=False):
    colnames=['variance', 'skewness', 'curtosis', 'entropy', 'y']
    if test:
        data = pd.read_csv('../data/bank-note/test.csv', header=None, names=colnames)
    else:
        data = pd.read_csv('../data/bank-note/train.csv', header=None, names=colnames)
    data['y'] = data['y'].apply(lambda x: -1 if x<1 else 1)
    if data_frame:
        return data
    else:
        X = data.drop('y', axis=1).values
        y = data['y'].values
        return X, y

In [3]:
def add_bias(X):
    bias = np.ones(X.shape[0]).reshape(-1, 1)
    X_fit = X.copy()
    X_fit = np.concatenate([bias, X_fit], axis=1)
    return X_fit

In [4]:
def pos_neg(x):
    return 1 if x > 0 else -1

In [5]:
def get_sign(y):
    return np.array(list(map(pos_neg, y)))

In [6]:
def get_accuracy(y_pred, y):
    return (y_pred == y).sum() / len(y)

In [7]:
def get_error(y_pred, y):
    return (y_pred != y).sum() / len(y)

## Get Data

In [8]:
X_train, y_train = read_bank_note(test=False)
X_test, y_test = read_bank_note(test=True)

## Standard

In [9]:
def fit_standard(X, y, T, lr):
    
    N, D = X.shape
    
    w = np.zeros(D)
    idx_order = np.arange(N)
    
    for i in range(T):
        shuffle(idx_order)
        X = X[idx_order]
        for b in range(N):

            xi = X[i]
            yi = y[i]

            pred = yi * w.T.dot(xi)

            if pred <= 0:
                w = w + lr * yi * xi

    return w

In [10]:
def predict_standard(X, w):
    
    y_pred = X.dot(w)
    y_pred = get_sign(y_pred)
    
    return y_pred

## Voted

In [11]:
def fit_voted(X, y, T, lr):
    
    N, D = X.shape
    
    w = np.zeros(D).reshape(1, -1)
    idx_order = np.arange(N)
    
    m=0
    C = np.array([0])
    
    for i in range(T):
        shuffle(idx_order)
        X = X[idx_order]
        for b in range(N):

            xi = X[i]
            yi = y[i]

            pred = yi * w[m].T.dot(xi)

            if pred <= 0:
                w_next = w[m].copy() + lr * yi * xi
                w = np.concatenate([w, w_next.reshape(1, -1)], axis=0)
                m += 1
                C = np.concatenate([C, np.array([1])])
                
            else:
                C[m] += 1

    return w, C

In [12]:
def predict_voted(X, w, C):
    
    y_pred = np.zeros(X.shape[0])
    
    for i in range(len(w)):
        sgn = X.dot(w[i])
        sgn = C[i] * get_sign(sgn)
        y_pred += sgn
        
    y_pred = get_sign(y_pred)
    return y_pred

## Averaged

In [13]:
def fit_averaged(X, y, T, lr):
    
    N, D = X.shape
    
    w = np.zeros(D)
    a = np.zeros(D)
    idx_order = np.arange(N)
    
    for i in range(T):
        shuffle(idx_order)
        X = X[idx_order]
        for b in range(N):

            xi = X[i]
            yi = y[i]

            pred = yi * w.T.dot(xi)

            if pred <= 0:
                w = w + lr * yi * xi
                
            a = a + w

    return a

## Perceptron

In [14]:
class Perceptron(object):
    
    def __init__(self, version='standard', lr=0.1):
        
        self.version = version
        self.lr = lr
        
    def fit(self, X, y, T=10):
        
        X_fit = add_bias(X)
        
        if self.version == 'standard':
            self.w = fit_standard(X_fit, y, T, self.lr)
        if self.version == 'voted':
            self.w, self.c = fit_voted(X_fit, y, T, self.lr)
        if self.version == 'averaged':
            self.w = fit_averaged(X_fit, y, T, self.lr)
            
        
    def predict(self, X):
        
        X_pred = add_bias(X)
        
        if self.version == 'standard':
            y_pred = predict_standard(X_pred, self.w)
        if self.version == 'voted':
            y_pred = predict_voted(X_pred, self.w, self.c)
        if self.version == 'averaged':
            y_pred = predict_standard(X_pred, self.w)
            
        return y_pred

## Results

In [45]:
lr = 0.01
T = 100

for ver in ['standard', 'voted', 'averaged']:
    model = Perceptron(version=ver, lr=lr)
    model.fit(X_train, y_train, T=T)
    y_pred = model.predict(X_train)
    print(ver.capitalize())
    print(f"Error:    {round(get_error(y_pred, y_train), 4)}")
    print(f"Accuracy: {round(get_accuracy(y_pred, y_train), 4)}\n")

Standard
Error:    0.6307
Accuracy: 0.3693

Voted
Error:    0.4014
Accuracy: 0.5986

Averaged
Error:    0.6124
Accuracy: 0.3876

